# Desafio para o processo seletivo Murabei

In [1]:
import pandas as pd

## DATA LOADING

In [2]:
# Carregando os dados das escolas
cat_school_data = pd.read_csv('cat_school_data.csv', sep=";")
num_school_data = pd.read_csv('num_school_data.csv', sep=";")

# Carregando os dados dos estudantes
cat_student_data = pd.read_csv('cat_student_data.csv', sep=";")
num_student_data = pd.read_csv('num_student_data.csv', sep=";")

# DADOS CATEGORICOS

# cat_student_data

In [3]:
# Dados referentes a escola
cat_student_data.head(5)

,school,student,variable,value
1,1,143,intake,bottom 25%
2,1,145,intake,mid 50%
3,1,142,intake,top 25%
4,1,141,intake,mid 50%
5,1,138,intake,mid 50%


In [4]:
print("'value' - {} valores diferentes \n'variable' - {} valores diferentes".format(cat_student_data['value'].nunique(), cat_student_data['variable'].nunique()))

'value' - 5 valores diferentes 
'variable' - 3 valores diferentes


Sendo eles:

In [5]:
cat_student_data['value'].value_counts()

mid 50%       4502
F             2353
bottom 25%    1750
top 25%       1657
M             1566
Name: value, dtype: int64

In [6]:
cat_student_data['variable'].value_counts()

vr        3962
intake    3947
sex       3919
Name: variable, dtype: int64

Agora, analisando as possiveis combinações, para cada escola,

In [7]:
cat_student_data.loc[(cat_student_data.variable=='vr')].groupby('school')['value'].nunique().value_counts()

1    65
Name: value, dtype: int64

In [8]:
cat_student_data.loc[(cat_student_data.variable=='vr')]['value'].value_counts()

mid 50%       2208
top 25%       1137
bottom 25%     617
Name: value, dtype: int64

Ou seja, para cada escola, só um um possivel 'value' de 'vr' (mid, top e bottom)

In [9]:
cat_student_data.loc[(cat_student_data.variable=='intake')].groupby('school')['value'].nunique().value_counts()

3    61
2     3
1     1
Name: value, dtype: int64

Ou seja, 61 escolas tem 3 tipos, 3 tem dois e 1 tem apenas 1.

Agora para varivel 'sex'

In [10]:
cat_student_data.loc[(cat_student_data.variable=='sex')]['value'].value_counts()

F    2353
M    1566
Name: value, dtype: int64

In [11]:
cat_student_data.loc[(cat_student_data.variable=='sex')].groupby('school')['value'].nunique().value_counts()

2    35
1    30
Name: value, dtype: int64

Apenas dois valores, como esperado.

# cat_school_data

In [12]:
cat_school_data.head(5)

,school,variable,value
0,1,schgend,mixed
1,2,schgend,girls
2,3,schgend,mixed
3,4,schgend,mixed
4,5,schgend,mixed


Aparentemente ets organizado como no caso anterior.

In [13]:
print("'value' - {} valores diferentes \n'variable' - {} valores diferentes".format(cat_school_data['value'].nunique(), cat_school_data['variable'].nunique()))

'value' - 5 valores diferentes 
'variable' - 2 valores diferentes


In [14]:
cat_school_data['variable'].value_counts()

type       65
schgend    65
Name: variable, dtype: int64

In [15]:
cat_school_data['value'].value_counts()

Mxd      35
mixed    35
Sngl     30
girls    20
boys     10
Name: value, dtype: int64

In [16]:
cat_school_data.loc[(cat_school_data.variable=='type')]['value'].value_counts()

Mxd     35
Sngl    30
Name: value, dtype: int64

In [17]:
cat_school_data.loc[(cat_school_data.variable=='schgend')]['value'].value_counts()

mixed    35
girls    20
boys     10
Name: value, dtype: int64

A varivel 'schgend' parece ser uma versão mais completa da variavel 'type'. Desta forma, será feito uma analise em relação a escola, com as seguintes variveis,

# schgend

In [18]:
cat_school_data_schgend = cat_school_data.loc[cat_school_data.variable=='schgend'][['school', 'value']]
cat_school_data_schgend.rename(columns={'value': 'moda_schgend'}, inplace=True)
cat_school_data_schgend.head(5)

,school,moda_schgend
0,1,mixed
1,2,girls
2,3,mixed
3,4,mixed
4,5,mixed


# vr

In [19]:
cat_school_data_vr = cat_student_data.loc[(cat_student_data.variable=='vr')].groupby('school')['value'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()
cat_school_data_vr.rename(columns={'value': 'moda_schgend'}, inplace=True)
cat_school_data_vr.head(5)

,school,moda_schgend
0,1,mid 50%
1,2,top 25%
2,3,top 25%
3,4,mid 50%
4,5,top 25%


# intake

In [20]:
cat_school_data_intake = cat_student_data.loc[(cat_student_data.variable=='intake')].groupby('school')['value'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()
cat_school_data_intake.rename(columns={'value': 'moda_intake'}, inplace=True)
cat_school_data_intake.head(5)

,school,moda_intake
0,1,mid 50%
1,2,mid 50%
2,3,bottom 25%
3,4,mid 50%
4,5,mid 50%


Somando tudo em apenas um dataframe

In [21]:
cat_student_base = pd.merge(cat_school_data_vr,
                            cat_school_data_intake,
                            on='school')

cat_student_base = pd.merge(cat_student_base, cat_school_data_schgend, on='school')
cat_student_base.head(5)



,school,moda_schgend_x,moda_intake,moda_schgend_y
0,1,mid 50%,mid 50%,mixed
1,2,top 25%,mid 50%,girls
2,3,top 25%,bottom 25%,mixed
3,4,mid 50%,mid 50%,mixed
4,5,top 25%,mid 50%,mixed


In [22]:
cat_data = pd.get_dummies(cat_student_base)
cat_data.head(5)

,school,moda_schgend_x_bottom 25%,moda_schgend_x_mid 50%,moda_schgend_x_top 25%,moda_intake_bottom 25%,moda_intake_mid 50%,moda_schgend_y_boys,moda_schgend_y_girls,moda_schgend_y_mixed
0,1,0,1,0,0,1,0,0,1
1,2,0,0,1,0,1,0,1,0
2,3,0,0,1,1,0,0,0,1
3,4,0,1,0,0,1,0,0,1
4,5,0,0,1,0,1,0,0,1


# DADOS NUMÉRICOS

In [23]:
num_student_data.head(5)

,school,student,variable,value
1,1,143.0,normexam,"0,2613242"
2,1,145.0,normexam,"0,1340672"
3,1,142.0,normexam,"-1,723882"
4,1,141.0,normexam,"0,9675862"
5,1,138.0,normexam,"0,5443412"


In [24]:
num_student_data['variable'].value_counts()

normexam    4059
standLRT    4059
Name: variable, dtype: int64

In [25]:
num_school_data.head(5)

,school,variable,value
0,1,schavg,"0,1661752"
1,2,schavg,"0,3951492"
2,3,schavg,"0,5141552"
3,4,schavg,"0,0917642"
4,5,schavg,"0,2105252"


In [26]:
num_school_data['variable'].value_counts()

schavg    65
Name: variable, dtype: int64

Sabendo da estrutura dos dados, será montada a seguinte função, que tem como objetivo selecionar uma varivel especifica, e agrupar por escolas.

In [27]:
def num_studant_school(num_student_data, variable_student='normexam'):

    # Filtrar os dados com base nas condições fornecidas
    num_student_data = num_student_data.loc[(num_student_data['variable'] == variable_student)]
    num_student_data = num_student_data[['school', 'value']]
    
    # Deletar todos os valores NA
    num_student_data = num_student_data.dropna()

    # Substituir vírgulas por pontos e converter para float
    num_student_data['mean_' + variable_student] = num_student_data['value'].str.replace(',', '.').astype(float)
    
    # Cálculo da média
    num_student_data_mean = num_student_data.groupby(['school']).mean().reset_index()

    return num_student_data_mean

Desta forma,

# normexam

Média de 'normexam' por escola - 'mean_normexam',


In [28]:
# normexam
student_normexam = num_studant_school(num_student_data, variable_student='normexam')
student_normexam.head(5)

,school,mean_normexam
0,1,0.509098
1,2,0.783102
2,3,0.877417
3,4,0.073629
4,5,0.384903


# standLRT
Média de 'standLRT' por escola - 'mean_standLRT',

In [29]:
# standLRT
student_standLRT = num_studant_school(num_student_data, variable_student='standLRT')
student_standLRT.head(5)

,school,mean_standLRT
0,1,0.166175
1,2,0.395149
2,3,0.514156
3,4,0.091764
4,5,0.210525


# schavg
A variável 'schavg' já está calculada por escola.

In [30]:
# schavg
num_school_data['schavg'] = num_school_data['value']
studant_schavg = num_school_data[['school', 'schavg']]
studant_schavg.head(5)

,school,schavg
0,1,"0,1661752"
1,2,"0,3951492"
2,3,"0,5141552"
3,4,"0,0917642"
4,5,"0,2105252"


Enfim, unindo as variveis para concluir o  dataset 'numeric_data',

In [31]:
# unindo os datasets
student_data = pd.merge(student_normexam, student_standLRT, on='school')
numeric_data = pd.merge(student_data, studant_schavg, on='school')


numeric_data.head(5)

,school,mean_normexam,mean_standLRT,schavg
0,1,0.509098,0.166175,"0,1661752"
1,2,0.783102,0.395149,"0,3951492"
2,3,0.877417,0.514156,"0,5141552"
3,4,0.073629,0.091764,"0,0917642"
4,5,0.384903,0.210525,"0,2105252"


Agora unindo os dataset numericos e categoricos, para efim chegar no 'data_base',

In [32]:
data_base = pd.merge(numeric_data, cat_data, on='school')
data_base.head(5)

,school,mean_normexam,mean_standLRT,schavg,moda_schgend_x_bottom 25%,moda_schgend_x_mid 50%,moda_schgend_x_top 25%,moda_intake_bottom 25%,moda_intake_mid 50%,moda_schgend_y_boys,moda_schgend_y_girls,moda_schgend_y_mixed
0,1,0.509098,0.166175,"0,1661752",0,1,0,0,1,0,0,1
1,2,0.783102,0.395149,"0,3951492",0,0,1,0,1,0,1,0
2,3,0.877417,0.514156,"0,5141552",0,0,1,1,0,0,0,1
3,4,0.073629,0.091764,"0,0917642",0,1,0,0,1,0,0,1
4,5,0.384903,0.210525,"0,2105252",0,0,1,0,1,0,0,1


Para responder a questão requisitada, vou usar a corrlação em relação a varivel 'mean_normexam'.

In [33]:
data_base.corrwith(data_base['mean_normexam']).sort_values()


moda_schgend_x_bottom 25%   -0.485559
moda_intake_mid 50%         -0.413242
moda_schgend_y_mixed        -0.210100
school                      -0.146439
moda_schgend_x_mid 50%      -0.091686
moda_schgend_y_boys         -0.024310
moda_schgend_y_girls         0.245937
moda_intake_bottom 25%       0.413242
moda_schgend_x_top 25%       0.548508
mean_standLRT                0.691641
mean_normexam                1.000000
dtype: float64

Analisando apenas a correlação, o módulo das escolas mistas e de meninas é praticamente o mesmo. Por sua vez, o das escolas de meninos é mais de 10 vezes menor. Assim, podemos concluir que, no caso das escolas de meninos, a influência geral em relação à média 'normexam' não é tão grande, mas nos outros casos (mista e de meninas) é.